# Numpy Broadcasting

이번에는 넘파이에서 매우 중요한 개념인 broadcasting에 대해서 알아볼 겁니다. Broadcasting은 tensorflow, keras, pytorch등 대부분의 딥러닝 라이브러리들도 채용하고 있는 개념이며, 사실상 numpy의 broadcasting과 동일합니다. 그리고, 딥러닝 코딩을 하실 때, broadcasting을 모른다면, 조금 힘들 수도 있습니다. 특히, tensorflow의 low level library(tensorflow v1 방식)를 사용하시고자 하신다면, broadcasting은 반드시 아셔야 할 개념입니다.

맨 먼저, numpy를 임포트하겠습니다.

In [2]:
import numpy as np

## Numpy Broadcasting

아주 중요한 파트입니다. Numpy의 broadcasting을 알고 모르고의 차이가, 코드의 양이나, 에러가 발생하고 안하고에 대해서 차이를 가저오기도 합니다.
Broadcasting은 element-wise연산에서 발생하는 현상으로, numpy는 기본적으로 서로 다른 모양의 텐서를 가지고는 element-wise 연산을 수행할 수 없습니다.
그렇지만, 다음 코드는 잘 실행됩니다.

In [5]:
A = np.random.randn(5, 3)
B = np.random.randn(5, 1)

print(f"A.shape: {A.shape}, B.shape: {B.shape}")
print()

C = A * B
print(C)
print(f"C.shape: {C.shape}")

A.shape: (5, 3), B.shape: (5, 1)

[[-1.64157666  0.30442519  0.9724483 ]
 [ 0.17028702  0.24914085 -0.15693847]
 [-0.12116754 -0.04302236  0.11239844]
 [ 0.09383733 -0.50060186  0.44908265]
 [ 1.25812037 -0.37862343  0.01893892]]
C.shape: (5, 3)


두 행렬 $A$, $B$는 서로 다른 차원을 가지고 있으며, 각각 $(5, 3)$, $(5, 1)$ 차원을 가집니다. 그리고, 이 둘을 element-wise 연산을 해 보니,
$(5, 3)$이 나왔습니다.

분명, 둘의 차원은 다른데, element-wise 곱이 이루어졌습니다.
이것은 numpy의 broadcasting이 수행되었기 때문인데요, broadcasting이란, 서로다른 차원의 행렬 또는 텐서를 element-wise 계산을 할 때, 작은 차원의 텐서를 stacking시켜서 큰 차원의 텐서의 차원으로 맞춰주는 것을 말합니다.

예를들어, 작은 차원 텐서인 $B$는 $(5, 1)$차원을 가집니다. 이놈을 출력해보면,

In [7]:
B

array([[ 1.23519979],
       [ 0.3780326 ],
       [ 0.11561806],
       [-0.34970919],
       [ 0.46752619]])

Numpy broadcasting은 $A$와 $B$의 차원을 맞춰주기 위해 이 $B$를 가로로 복제합니다. 예를들면, 다음과 같은 형식인데, ``np.tile``함수는 신경쓰지 마세요. 다만, broadcasting이 이 함수와 매우 유사한 역할을 합니다. 똑같은 열벡터가 3개로 복제된 것에 주목해주시길 바랍니다.

In [8]:
np.tile(B, (1, 3))

array([[ 1.23519979,  1.23519979,  1.23519979],
       [ 0.3780326 ,  0.3780326 ,  0.3780326 ],
       [ 0.11561806,  0.11561806,  0.11561806],
       [-0.34970919, -0.34970919, -0.34970919],
       [ 0.46752619,  0.46752619,  0.46752619]])

그리고 나서, $A * B$를 수행하게 됩니다.

In [9]:
A * np.tile(B, (1, 3))

array([[-1.64157666,  0.30442519,  0.9724483 ],
       [ 0.17028702,  0.24914085, -0.15693847],
       [-0.12116754, -0.04302236,  0.11239844],
       [ 0.09383733, -0.50060186,  0.44908265],
       [ 1.25812037, -0.37862343,  0.01893892]])

In [10]:
A * B

array([[-1.64157666,  0.30442519,  0.9724483 ],
       [ 0.17028702,  0.24914085, -0.15693847],
       [-0.12116754, -0.04302236,  0.11239844],
       [ 0.09383733, -0.50060186,  0.44908265],
       [ 1.25812037, -0.37862343,  0.01893892]])

즉, broadcasting이란, 서로다른 크기의 두 텐서를 element-wise 연산을 수행할 때, 작은 차원의 텐서를 특정 축으로 복제해서 쌓고, 차원을 맞춰준 후 연산을 수행하는 것을 말합니다.
Numpy의 강력한 기능이죠.

몇 가지 차원 예제를 들어보면,

(4, 3) op (3, 1)       -> ERROR! 첫 번쨰 차원이 다른데 1이 없습니다. 따라서 어떻게 복사해야 할지 모릅니다. 에러.  
(4, 1) op (1, 3)       -> ERROR! 이건 첫번쨰 배열을 복사하고, 두번째를.. 어느순서로 해야하죠? 에러.  
(4, 3, 2) op (4, 1, 2) -> 두 번째 배열이 axis=1에서 브로드캐스팅되어 연산됩니다. 결과 배열의 차원은 (4, 3, 2).  
(5, 2, 3) op (1, 2, 1) -> 두 번째 배열이 axis=(0, 2)에서 브로드캐스팅됩니다. 결과 배열의 차원은 (5, 2, 3).  
(1, 1, 1) op (1, 4, 5) -> 첫 번째 배열이 axis=(1, 2)에서 브로드캐스팅됩니다. 결과 배열의 차원은 (1, 4, 5).  
(4, 3) op (3,)         -> ERROR!두 배열의 rank가 다름. 에러  
() op (4, 2, 2)        -> 첫 번째 피연산자가 넘파이 배열이 아닌 파이썬 숫자임. 이 경우, 이 숫자를 (4, 2, 2)차원으로 싹다 복사해서 두번째 배열과 연산함. 결과 차원은  (4, 2, 2)